In [5]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import pytorch_lightning as pl
from src.seq2conn.models.simple_vae import SimpleVAE
from src.seq2conn.data import RandomDataModule
from src.seq2conn.models.dgmg import DGMG

In [6]:
DGMG(.2, 10, 5)

DGMG(
  (graph_embed): GraphEmbed(
    (node_gating): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Sigmoid()
    )
    (node_to_graph): Linear(in_features=10, out_features=20, bias=True)
  )
  (graph_prop): GraphProp(
    (message_funcs): ModuleList(
      (0-4): 5 x Linear(in_features=21, out_features=20, bias=True)
    )
    (node_update_funcs): ModuleList(
      (0-4): 5 x GRUCell(20, 10)
    )
  )
  (add_node_agent): AddNode(
    (add_node): Linear(in_features=20, out_features=1, bias=True)
    (node_type_embed): Embedding(1, 10)
    (initialize_hv): Linear(in_features=30, out_features=10, bias=True)
  )
  (add_edge_agent): AddEdge(
    (add_edge): Linear(in_features=30, out_features=1, bias=True)
  )
  (choose_dest_agent): ChooseDestAndUpdate(
    (choose_dest): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [ ]:
datamodule = RandomDataModule(batch_size=16, n_timesteps=127, input_dim=18)


model = SimpleVAE(18, 12, 2)

trainer = pl.Trainer(max_epochs=100, log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)